In [1]:
import os
import pandas as pd
import cobra
import medusa

from cobra.core import Reaction
from medusa.reconstruct.expand.expand import iterative_gapfill_from_binary_phenotypes


/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def load_universal_modelseed():
    seed_rxn_table = pd.read_csv('../data/reactions_seed_20180809.tsv',sep='\t')
    seed_rxn_table['id'] = seed_rxn_table['id'] + '_c'
    universal = cobra.io.load_json_model('../data/universal_mundy.json')
    # remove any reactions from the universal that don't have "OK" status
    # in modelSEED (guards against mass and charge-imbalanced reactions)
    ok_ids = list(seed_rxn_table.loc[(seed_rxn_table['status'] == 'OK') | (seed_rxn_table['status'] == 'HB')]['id'])
    remove_rxns = []
    for reaction in universal.reactions:
        if reaction.id not in ok_ids:
            remove_rxns.append(reaction)
    universal.remove_reactions(remove_rxns)
    # remove metabolites from the universal that are no longer present in any
    # reactions.
    mets_in_reactions = []
    for reaction in universal.reactions:
        mets = [met.id for met in reaction.metabolites]
        mets_in_reactions.extend(mets)
    mets_in_reactions = set(mets_in_reactions)

    mets_missing_reactions = []
    for metabolite in universal.metabolites:
        if metabolite.id not in mets_in_reactions:
            mets_missing_reactions.append(metabolite)
    universal.remove_metabolites(mets_missing_reactions)

    universal.repair()
    return universal


In [3]:
master_universal = load_universal_modelseed()

In [ ]:
# Load the biolog composition to be used for gapfilling
biolog_base_composition = pd.read_csv('../data/biolog_base_composition.csv',sep=',')
biolog_base_dict = dict(zip(biolog_base_composition['ID'],\
                          [1000 for i in range(0,len(biolog_base_composition['ID']))]))
# The biolog growth file has already been filtered by species that meet
# the minimum carbon source requirement, so we can use the entire dataframe
biolog_thresholded = pd.read_csv('../data/plata_thresholded.csv',sep='\t',index_col=0)

# get the list of ensembles already generated.
already_generated = os.listdir('../results/deep_ensembles/')
# remove the .json extension to just get the name for each species
already_generated = [s.split('.')[0] for s in already_generated]

# Exclude species for which there is no feasible solution
# using this reaction bag (identified during previous iterations
# of this analysis)
exclude_species = ["Brachybacterium faecium"]


# Iterate over each species and generate and ensemble for each
for species_file in os.listdir('../data/modelseed_models/'): 
    
    # Load the species model. only continue if the species is in the filtered
    # biolog dataframe (i.e. it met our filtering criteria)
    species_name = species_file.split('.')[0]
    if (species_name in biolog_thresholded.index) and (
        species_name not in already_generated) and (
        species_name not in exclude_species):
        print("Building ensemble for " + species_name)
        model = cobra.io.load_json_model('../data/modelseed_models/' + species_file)

        # extract the biolog conditions for the model of interest
        mod_pheno = biolog_thresholded.loc[species_name]
        mod_pheno = list(mod_pheno[mod_pheno == True].index)

        # generate a fresh universal for each species
        universal = master_universal.copy()

        # check for biolog base components in the model. Add exchange reactions
        # if none exist and add the metabolite to the model if it does not
        # already exist
        add_mets = []
        add_exchanges = []
        for met in list(biolog_base_dict.keys()):
            try:
                model.metabolites.get_by_id(met)
            except:
                print('no '+met)
                add_met = universal.metabolites.get_by_id(met).copy()
                add_mets.append(add_met)

        model.add_metabolites(add_mets)

        for met in list(biolog_base_dict.keys()):
            # Search for exchange reactions
            try:
                model.reactions.get_by_id('EX_'+met)
            except:
                add_met = model.metabolites.get_by_id(met)
                ex_rxn = Reaction('EX_' + met)
                ex_rxn.name = "Exchange reaction for " + met
                ex_rxn.lower_bound = -1000
                ex_rxn.upper_bound = 1000
                ex_rxn.add_metabolites({add_met:-1})
                add_exchanges.append(ex_rxn)

        model.add_reactions(add_exchanges)

        # Find metabolites from the biolog data that are missing in the model
        # and add them from the universal
        missing_mets = []
        missing_exchanges = []
        media_dicts = {}
        for met_id in mod_pheno:
            try:
                model.metabolites.get_by_id(met_id)
            except:
                print(met_id + " was not in model, adding met and exchange reaction")
                met = universal.metabolites.get_by_id(met_id).copy()
                missing_mets.append(met)
                ex_rxn = Reaction('EX_' + met_id)
                ex_rxn.name = "Exchange reaction for " + met_id
                ex_rxn.lower_bound = -1000
                ex_rxn.upper_bound = 1000
                ex_rxn.add_metabolites({met:-1})
                missing_exchanges.append(ex_rxn)
            media_dicts[met_id] = biolog_base_dict.copy()
            media_dicts[met_id] = {'EX_'+k:v for k,v in media_dicts[met_id].items()}
            media_dicts[met_id]['EX_'+met_id] = 1000
        model.add_metabolites(missing_mets)
        model.add_reactions(missing_exchanges)

        # identify transporters for each biolog component in the universal model
        # and pick one that will enable transport in the gapfilling problem.
        transporters_in_universal = [rxn for rxn in universal.reactions if len(rxn.compartments)>1]
        for met in media_dicts.keys():
            metabolite = model.metabolites.get_by_id(met)
            base_met_id = met.split('_')[0]
            rxns_with_metabolite = metabolite.reactions
            transport = False
            for rxn in rxns_with_metabolite:
                metabolites = [met_in_rxn.id for met_in_rxn in rxn.metabolites]
                if (base_met_id+'_e' in metabolites and base_met_id+'_c' in metabolites):
                    transport = True

            pick_transporter = {}
            if not transport:
                print("missing transporter for " + metabolite.name)
                for rxn in transporters_in_universal:
                    metabolites = [met_in_rxn.id for met_in_rxn in rxn.metabolites]
                    if (base_met_id+'_e' in metabolites and base_met_id+'_c' in metabolites):
                        pick_transporter[met] = rxn.id

        # Add the transporters to the model
        transporters_to_add = list(pick_transporter.values())
        transporter_list = []
        for rxn in transporters_to_add:
            transporter_list.append(universal.reactions.get_by_id(rxn).copy())
        model.add_reactions(transporter_list)

        # remove the added transporters from the universal model
        universal.remove_reactions([universal.reactions.get_by_id(rxn) for rxn in transporters_to_add])

        # generate the ensemble for this species
        num_cycles = 1000
        lower_bound = 0.05
        ensemble = iterative_gapfill_from_binary_phenotypes(\
                                         model,\
                                         universal,\
                                         media_dicts,\
                                         num_cycles,\
                                         lower_bound=lower_bound,\
                                         inclusion_threshold=1E-11,\
                                         exchange_reactions=False,\
                                         demand_reactions=False,\
                                         exchange_prefix='EX')

        # save the ensemble by pickling it
        ensemble.to_pickle('../results/deep_ensembles/'+species_name+'.pickle')

Building ensemble for Bacillus megaterium
no cpd10515_e
cpd11594_e was not in model, adding met and exchange reaction
cpd03845_e was not in model, adding met and exchange reaction
cpd05158_e was not in model, adding met and exchange reaction
cpd01133_e was not in model, adding met and exchange reaction
cpd00382_e was not in model, adding met and exchange reaction
cpd00208_e was not in model, adding met and exchange reaction
cpd15584_e was not in model, adding met and exchange reaction
cpd00492_e was not in model, adding met and exchange reaction
cpd03019_e was not in model, adding met and exchange reaction
cpd00232_e was not in model, adding met and exchange reaction
cpd19001_e was not in model, adding met and exchange reaction
cpd00709_e was not in model, adding met and exchange reaction
cpd00750_e was not in model, adding met and exchange reaction
cpd00751_e was not in model, adding met and exchange reaction
cpd00396_e was not in model, adding met and exchange reaction
cpd01307_e was

starting cycle number 160
starting cycle number 161
starting cycle number 162
starting cycle number 163
starting cycle number 164
starting cycle number 165
starting cycle number 166
starting cycle number 167
starting cycle number 168
starting cycle number 169
starting cycle number 170
starting cycle number 171
starting cycle number 172
starting cycle number 173
starting cycle number 174
starting cycle number 175
starting cycle number 176
starting cycle number 177
starting cycle number 178
starting cycle number 179
starting cycle number 180
starting cycle number 181
starting cycle number 182
starting cycle number 183
starting cycle number 184
starting cycle number 185
starting cycle number 186
starting cycle number 187
starting cycle number 188
starting cycle number 189
starting cycle number 190
starting cycle number 191
starting cycle number 192
starting cycle number 193
starting cycle number 194
starting cycle number 195
starting cycle number 196
starting cycle number 197
starting cyc

starting cycle number 476
starting cycle number 477
starting cycle number 478
starting cycle number 479
starting cycle number 480
starting cycle number 481
starting cycle number 482
starting cycle number 483
starting cycle number 484
starting cycle number 485
starting cycle number 486
starting cycle number 487
starting cycle number 488
starting cycle number 489
starting cycle number 490
starting cycle number 491
starting cycle number 492
starting cycle number 493
starting cycle number 494
starting cycle number 495
starting cycle number 496
starting cycle number 497
starting cycle number 498
starting cycle number 499
starting cycle number 500
starting cycle number 501
starting cycle number 502
starting cycle number 503
starting cycle number 504
starting cycle number 505
starting cycle number 506
starting cycle number 507
starting cycle number 508
starting cycle number 509
starting cycle number 510
starting cycle number 511
starting cycle number 512
starting cycle number 513
starting cyc

starting cycle number 792
starting cycle number 793
starting cycle number 794
starting cycle number 795
starting cycle number 796
starting cycle number 797
starting cycle number 798
starting cycle number 799
starting cycle number 800
starting cycle number 801
starting cycle number 802
starting cycle number 803
starting cycle number 804
starting cycle number 805
starting cycle number 806
starting cycle number 807
starting cycle number 808
starting cycle number 809
starting cycle number 810
starting cycle number 811
starting cycle number 812
starting cycle number 813
starting cycle number 814
starting cycle number 815
starting cycle number 816
starting cycle number 817
starting cycle number 818
starting cycle number 819
starting cycle number 820
starting cycle number 821
starting cycle number 822
starting cycle number 823
starting cycle number 824
starting cycle number 825
starting cycle number 826
starting cycle number 827
starting cycle number 828
starting cycle number 829
starting cyc

starting cycle number 53
starting cycle number 54
starting cycle number 55
starting cycle number 56
starting cycle number 57
starting cycle number 58
starting cycle number 59
starting cycle number 60
starting cycle number 61
starting cycle number 62
starting cycle number 63
starting cycle number 64
starting cycle number 65
starting cycle number 66
starting cycle number 67
starting cycle number 68
starting cycle number 69
starting cycle number 70
starting cycle number 71
starting cycle number 72
starting cycle number 73
starting cycle number 74
starting cycle number 75
starting cycle number 76
starting cycle number 77
starting cycle number 78
starting cycle number 79
starting cycle number 80
starting cycle number 81
starting cycle number 82
starting cycle number 83
starting cycle number 84
starting cycle number 85
starting cycle number 86
starting cycle number 87
starting cycle number 88
starting cycle number 89
starting cycle number 90
starting cycle number 91
starting cycle number 92


...

starting cycle number 378
starting cycle number 379
starting cycle number 380
starting cycle number 381
starting cycle number 382
starting cycle number 383
starting cycle number 384
starting cycle number 385
starting cycle number 386
starting cycle number 387
starting cycle number 388
starting cycle number 389
starting cycle number 390
starting cycle number 391
starting cycle number 392
starting cycle number 393
starting cycle number 394
starting cycle number 395
starting cycle number 396
starting cycle number 397
starting cycle number 398
starting cycle number 399
starting cycle number 400
starting cycle number 401
starting cycle number 402
starting cycle number 403
starting cycle number 404
starting cycle number 405
starting cycle number 406
starting cycle number 407
starting cycle number 408
starting cycle number 409
starting cycle number 410
starting cycle number 411
starting cycle number 412
starting cycle number 413
starting cycle number 414
starting cycle number 415
starting cyc

starting cycle number 694
starting cycle number 695
starting cycle number 696
starting cycle number 697
starting cycle number 698
starting cycle number 699
starting cycle number 700
starting cycle number 701
starting cycle number 702
starting cycle number 703
starting cycle number 704
starting cycle number 705
starting cycle number 706
starting cycle number 707
starting cycle number 708
starting cycle number 709
starting cycle number 710
starting cycle number 711
starting cycle number 712
starting cycle number 713
starting cycle number 714
starting cycle number 715
starting cycle number 716
starting cycle number 717
starting cycle number 718
starting cycle number 719
starting cycle number 720
starting cycle number 721
starting cycle number 722
starting cycle number 723
starting cycle number 724
starting cycle number 725
starting cycle number 726
starting cycle number 727
starting cycle number 728
starting cycle number 729
starting cycle number 730
starting cycle number 731
starting cyc

Constraining lower bound for bio1
starting cycle number 0
starting cycle number 1
starting cycle number 2
starting cycle number 3
starting cycle number 4
starting cycle number 5
starting cycle number 6
starting cycle number 7
starting cycle number 8
starting cycle number 9
starting cycle number 10
starting cycle number 11
starting cycle number 12
starting cycle number 13
starting cycle number 14
starting cycle number 15
starting cycle number 16
starting cycle number 17
starting cycle number 18
starting cycle number 19
starting cycle number 20
starting cycle number 21
starting cycle number 22
starting cycle number 23
starting cycle number 24
starting cycle number 25
starting cycle number 26
starting cycle number 27
starting cycle number 28
starting cycle number 29
starting cycle number 30
starting cycle number 31
starting cycle number 32
starting cycle number 33
starting cycle number 34
starting cycle number 35
starting cycle number 36
starting cycle number 37
starting cycle number 38
s

starting cycle number 319
starting cycle number 320
starting cycle number 321
starting cycle number 322
starting cycle number 323
starting cycle number 324
starting cycle number 325
starting cycle number 326
starting cycle number 327
starting cycle number 328
starting cycle number 329
starting cycle number 330
starting cycle number 331
starting cycle number 332
starting cycle number 333
starting cycle number 334
starting cycle number 335
starting cycle number 336
starting cycle number 337
starting cycle number 338
starting cycle number 339
starting cycle number 340
starting cycle number 341
starting cycle number 342
starting cycle number 343
starting cycle number 344
starting cycle number 345
starting cycle number 346
starting cycle number 347
starting cycle number 348
starting cycle number 349
starting cycle number 350
starting cycle number 351
starting cycle number 352
starting cycle number 353
starting cycle number 354
starting cycle number 355
starting cycle number 356
starting cyc

starting cycle number 635
starting cycle number 636
starting cycle number 637
starting cycle number 638
starting cycle number 639
starting cycle number 640
starting cycle number 641
starting cycle number 642
starting cycle number 643
starting cycle number 644
starting cycle number 645
starting cycle number 646
starting cycle number 647
starting cycle number 648
starting cycle number 649
starting cycle number 650
starting cycle number 651
starting cycle number 652
starting cycle number 653
starting cycle number 654
starting cycle number 655
starting cycle number 656
starting cycle number 657
starting cycle number 658
starting cycle number 659
starting cycle number 660
starting cycle number 661
starting cycle number 662
starting cycle number 663
starting cycle number 664
starting cycle number 665
starting cycle number 666
starting cycle number 667
starting cycle number 668
starting cycle number 669
starting cycle number 670
starting cycle number 671
starting cycle number 672
starting cyc